In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.svm import OneClassSVM 
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import itertools
import pickle

In [3]:
df = pd.read_csv("../data/run-over-dataset.csv")
print(df.shape)

columns_to_drop = ['VERBALE', 'DATA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro',
                    'Totale', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino', 'I costa dx', 'II costa dx',
                    'III costa dx', 'IV costa dx', 'V costa dx', 'VI costa dx', 'VII costa dx', 
                    'VIII costa dx', 'IX costa dx', 'X costa dx', 'XI costa dx', 'XII costa dx',
                    'I costa sx', 'II costa sx', 'III costa sx', 'IV costa sx', 'V costa sx', 
                    'VI costa sx', 'VII costa sx', 'VIII costa sx', 'IX costa sx', 
                    'X costa sx', 'XI costa sx', 'XII costa sx']

X = df.drop(columns=columns_to_drop)
print(X.shape)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 367)
(130, 326)
(130, 274)


Inizialmente sono state eliminate le colonne che fanno riferimento ai totali delle varie zono del corpo, in quanto sono una combinazione lineare delle precedenti colonne che sono già presenti all'interno del data set.
Poi sono state eliminate le colonne a valori costanti in quanto non sono significative per la classificazione.
Sono state, infine, rimosse le colonne duplicate.
Da 367 colonne colonne iniziale, ora il data set è formato da 274 colonne.

In [4]:
def nested_cv_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_accuracy = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(kernels, gammas, nus):
                    clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    accuracy = accuracy_score(true_values, pred_values)
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [5]:
results = nested_cv_accuracy(X)
results

{'algorythm': 'OneClassSVM',
 'best kernel': 'rbf',
 'best gamma': 1e-09,
 'best nu': 0.56,
 'method used for model selection': 'accuracy on nested cv',
 'accuracy mean': 56.46153846153846,
 'accuracy std': 8.649657105766522,
 'precision mean': 64.6088409956057,
 'precision  std': 30.8541312246178,
 'recall mean': 33.142857142857146,
 'recall std': 26.137782489518653,
 'data set': 'no const, no equals, no tot, standard scaled'}

In [6]:
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.56,accuracy on nested cv,56.461538,8.649657,64.608841,30.854131,33.142857,26.137782,"no const, no equals, no tot, standard scaled"


In [7]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [8]:
def nested_cv_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_f1 = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(kernels, gammas, nus):
                    clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    f1 = f1_score(true_values, pred_values)
                    if f1 > best_f1:
                        best_f1 = f1
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [9]:
scores_df = add_record(scores_df, nested_cv_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.56,accuracy on nested cv,56.461538,8.649657,64.608841,30.854131,33.142857,26.137782,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-06,0.54,f1 on nested cv,58.384615,8.623966,64.632238,25.916988,44.142857,27.281226,"no const, no equals, no tot, standard scaled"


In [10]:
def outer_cv_inner_holdout_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

            best_accuracy = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [11]:
scores_df = add_record(scores_df, outer_cv_inner_holdout_accuracy(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.56,accuracy on nested cv,56.461538,8.649657,64.608841,30.854131,33.142857,26.137782,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-06,0.54,f1 on nested cv,58.384615,8.623966,64.632238,25.916988,44.142857,27.281226,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.47,accuracy on outer cv inner holdout,53.615385,8.370489,60.119383,36.653776,22.000000,21.798708,"no const, no equals, no tot, standard scaled"


In [12]:
def outer_cv_inner_holdout_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

            best_f1 = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [13]:
scores_df = add_record(scores_df, outer_cv_inner_holdout_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.56,accuracy on nested cv,56.461538,8.649657,64.608841,30.854131,33.142857,26.137782,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-06,0.54,f1 on nested cv,58.384615,8.623966,64.632238,25.916988,44.142857,27.281226,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.47,accuracy on outer cv inner holdout,53.615385,8.370489,60.119383,36.653776,22.000000,21.798708,"no const, no equals, no tot, standard scaled"
3,OneClassSVM,rbf,1.000000e-09,0.70,f1 on outer cv inner holdout,56.769231,9.501168,58.153908,30.097213,40.857143,29.278396,"no const, no equals, no tot, standard scaled"


In [14]:
def outer_holdout_inner_cv_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_accuracy = 0

        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [15]:
scores_df = add_record(scores_df, outer_holdout_inner_cv_accuracy(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.56,accuracy on nested cv,56.461538,8.649657,64.608841,30.854131,33.142857,26.137782,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-06,0.54,f1 on nested cv,58.384615,8.623966,64.632238,25.916988,44.142857,27.281226,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.47,accuracy on outer cv inner holdout,53.615385,8.370489,60.119383,36.653776,22.000000,21.798708,"no const, no equals, no tot, standard scaled"
3,OneClassSVM,rbf,1.000000e-09,0.70,f1 on outer cv inner holdout,56.769231,9.501168,58.153908,30.097213,40.857143,29.278396,"no const, no equals, no tot, standard scaled"
4,OneClassSVM,rbf,1.000000e-09,0.03,accuracy on outer holdout inner cv,55.000000,13.439542,67.584500,39.244976,30.917529,23.236898,"no const, no equals, no tot, standard scaled"


In [16]:
def outer_holdout_inner_cv_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_f1 = 0

        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [17]:
scores_df = add_record(scores_df, outer_holdout_inner_cv_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.56,accuracy on nested cv,56.461538,8.649657,64.608841,30.854131,33.142857,26.137782,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-06,0.54,f1 on nested cv,58.384615,8.623966,64.632238,25.916988,44.142857,27.281226,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.47,accuracy on outer cv inner holdout,53.615385,8.370489,60.119383,36.653776,22.000000,21.798708,"no const, no equals, no tot, standard scaled"
3,OneClassSVM,rbf,1.000000e-09,0.70,f1 on outer cv inner holdout,56.769231,9.501168,58.153908,30.097213,40.857143,29.278396,"no const, no equals, no tot, standard scaled"
4,OneClassSVM,rbf,1.000000e-09,0.03,accuracy on outer holdout inner cv,55.000000,13.439542,67.584500,39.244976,30.917529,23.236898,"no const, no equals, no tot, standard scaled"
5,OneClassSVM,linear,1.000000e-09,0.42,f1 on outer holdout inner cv,56.153846,8.104349,72.333333,17.495237,36.671243,13.842532,"no const, no equals, no tot, standard scaled"


In [18]:
def double_holdout_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_accuracy = 0
        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(kernels, gammas, nus):
            clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            accuracy = accuracy_score(true_values, pred_values)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['kernel'] = params[0]
                best_params['gamma'] = params[1]
                best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [19]:
scores_df = add_record(scores_df, double_holdout_accuracy(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.56,accuracy on nested cv,56.461538,8.649657,64.608841,30.854131,33.142857,26.137782,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-06,0.54,f1 on nested cv,58.384615,8.623966,64.632238,25.916988,44.142857,27.281226,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.47,accuracy on outer cv inner holdout,53.615385,8.370489,60.119383,36.653776,22.000000,21.798708,"no const, no equals, no tot, standard scaled"
3,OneClassSVM,rbf,1.000000e-09,0.70,f1 on outer cv inner holdout,56.769231,9.501168,58.153908,30.097213,40.857143,29.278396,"no const, no equals, no tot, standard scaled"
4,OneClassSVM,rbf,1.000000e-09,0.03,accuracy on outer holdout inner cv,55.000000,13.439542,67.584500,39.244976,30.917529,23.236898,"no const, no equals, no tot, standard scaled"
5,OneClassSVM,linear,1.000000e-09,0.42,f1 on outer holdout inner cv,56.153846,8.104349,72.333333,17.495237,36.671243,13.842532,"no const, no equals, no tot, standard scaled"
6,OneClassSVM,linear,1.000000e-09,0.64,accuracy on double holdout,55.384615,8.284869,78.000000,30.265492,23.025279,16.794986,"no const, no equals, no tot, standard scaled"


In [20]:
def double_holdout_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_f1 = 0
        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(kernels, gammas, nus):
            clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            f1 = f1_score(true_values, pred_values)
            if f1 > best_f1:
                best_f1 = f1
                best_params['kernel'] = params[0]
                best_params['gamma'] = params[1]
                best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [21]:
scores_df = add_record(scores_df, double_holdout_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.56,accuracy on nested cv,56.461538,8.649657,64.608841,30.854131,33.142857,26.137782,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-06,0.54,f1 on nested cv,58.384615,8.623966,64.632238,25.916988,44.142857,27.281226,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.47,accuracy on outer cv inner holdout,53.615385,8.370489,60.119383,36.653776,22.000000,21.798708,"no const, no equals, no tot, standard scaled"
3,OneClassSVM,rbf,1.000000e-09,0.70,f1 on outer cv inner holdout,56.769231,9.501168,58.153908,30.097213,40.857143,29.278396,"no const, no equals, no tot, standard scaled"
4,OneClassSVM,rbf,1.000000e-09,0.03,accuracy on outer holdout inner cv,55.000000,13.439542,67.584500,39.244976,30.917529,23.236898,"no const, no equals, no tot, standard scaled"
5,OneClassSVM,linear,1.000000e-09,0.42,f1 on outer holdout inner cv,56.153846,8.104349,72.333333,17.495237,36.671243,13.842532,"no const, no equals, no tot, standard scaled"
6,OneClassSVM,linear,1.000000e-09,0.64,accuracy on double holdout,55.384615,8.284869,78.000000,30.265492,23.025279,16.794986,"no const, no equals, no tot, standard scaled"
7,OneClassSVM,rbf,1.000000e-09,0.50,f1 on double holdout,66.153846,11.383576,75.841600,18.183735,52.582001,22.814115,"no const, no equals, no tot, standard scaled"


In [22]:
file_path = 'svm_exp3_df.pickle'

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)